In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
from collections import defaultdict
import re
import itertools

workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

In [2]:
# get files and check if years are same for both sensors. Also check what indices and metrics are available
training_files = getFilelist('/data/Aldhani/eoagritwin/et/Training_ML/training_data/', '.parquet')

indices_by_year = defaultdict(set)

for file in training_files:
 
    match  = re.search(r'/(S2|S3)_(\d{4})(?:_([A-Za-z]{3,10}))?', file) # (?:_([A-Z]{3}))? --> the ? at the end makes the whole group optional, i.e. also files where 
                                                                    # only the first 2 groups match will be considered. the first question mark stops the loop from breaking,
                                                                    # if a file with only the first 2 groups is encountered
    if match:
        sensor, year, index = match.groups()
        if index:  # Only S2 has metrics in filename
            indices_by_year[year].add(f'{sensor}:{index}')
    

# Get all unique indices
all_indices = sorted({index for indices in indices_by_year.values() for index in indices})

# Prepare a table: rows = years, columns = indices
table_data = []
for year in sorted(indices_by_year):
    row = {index: '✔' if index in indices_by_year[year] else 'X' for index in all_indices}
    row['Year'] = year
    table_data.append(row)

# Create and show DataFrame
df = pd.DataFrame(table_data)
df = df.set_index('Year')
print('\nDatasets available for training\n')
print(df)



Datasets available for training

     S2:EVI S2:NDM S2:NDV S2:TCB S2:TCG S2:TCW S3:mean S3:median
Year                                                            
2017      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔
2018      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔
2019      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔
2020      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔
2021      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔
2022      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔
2023      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔
2024      ✔      ✔      ✔      ✔      ✔      ✔       ✔         ✔


In [3]:
# bring S2 together and in shape
index_conti = []
for index in [col.split(':')[-1] for col in df.columns[:-2]]:
    print(index)
    temp_conti = []
    for file in training_files:
        if index in file:
            print(file)
            dat = pd.read_parquet(file)
            dat = dat.assign(year = re.search(r'_(\d{4})_', file).group(1))
            dat = dat[['year', 'doy', 'row', 'col', 'S2', 'index']]
            dat = dat.rename(columns={'S2': file.split('_')[-1].split('.')[0]})
            dat = dat.drop('index', axis=1)
            dat.dropna()
            temp_conti.append(dat)
    index_conti.append(pd.concat(temp_conti, ignore_index=False))

EVI
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2017_EVI.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2018_EVI.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2019_EVI.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2020_EVI.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2021_EVI.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2022_EVI.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2023_EVI.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2024_EVI.parquet
NDM
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2017_NDM.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2018_NDM.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2019_NDM.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2020_NDM.parquet
/data/Aldhani/eoagritwin/et/Training_ML/training_data/S2_2021_NDM.parquet
/data/Aldhani/eoagritwin/et/Tr

In [ ]:
S2_block = index_conti[0]
for i in range(1, len(index_conti)):
    print(i)
    S2_block = pd.merge(S2_block, index_conti[i], on=['row', 'col', 'doy', 'year'], how='inner')

1


In [ ]:
S2_wide.to_parquet(f'/data/{origin}et/Training_ML/training_data/S3_powerblock.parquet', index=False)

In [ ]:
result = []
S2_years = [str(year) for year in range(2017,2025)]
S2_indices = ['EVI', 'NDM', 'NDV', 'TCB', 'TCG', 'TCW']

In [ ]:
for r1 in range(1, len(S2_years) + 1):
    for combo1 in itertools.combinations(S2_years, r1):
        prefix = ' '.join(combo1)

        # For each, combine with all non-empty combinations of list2
        for r2 in range(1, len(S2_indices) + 1):
            for combo2 in itertools.combinations(S2_indices, r2):
                suffix = ' + '.join(combo2)
                result.append(f'{prefix} with {suffix}')


In [ ]:
[file for file in training_files if any(substr in file for substr in S2_indices[2:4]) and any(substr in file for substr in S2_years[2:4])]

In [ ]:
indices_by_year